# Import and Define

data can be download from https://drive.google.com/file/d/1_xubS1Emt9UwnMIJVUSKxg8OsYVDcXWW/view?usp=sharing

In [49]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import itertools
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Lasso
from patsy import dmatrices

In [50]:
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, r2 
def formula(formula_without_y,y=''):
    formula_y = y+formula_without_y
    formula_y = formula_y.replace('+' + y, '')
    formula_final = formula_y
    return formula_final

In [51]:
#Importing the dataset into a dataframe
raw_data = pd.read_csv("Financials_with_IR.csv")

In [52]:
#Printing the first five rows
raw_data.head()

,Unnamed: 0,NIMY,IDT1RWAJR,LNATRES,ASSTLTR,RWAJ,SZ100T5,STNAME,INTINC,IGLSECR,...,EINTEXPA,LIAB,BRO,NIMQR,CALLYM,EQTOT,ID,YEAR,AVG_IR,DELTA_AR
0,0,4.617378,17.696268,490,40.814425,45066.0,0,ALABAMA,5083,0.193805,...,2068.0,65199,0,4.552379,200112,7785,13887_20011231,2001,3.501522,-0.825182
1,1,3.869881,17.525382,623,42.124781,65893.0,0,ALABAMA,6657,0.308600,...,3385.0,80400,0,3.807386,200112,11553,14184_20011231,2001,3.501522,-0.825182
2,2,4.461363,11.167648,1601,32.907955,142483.0,1,ALABAMA,15120,0.357381,...,7338.0,186332,4468,4.191435,200112,16233,14720_20011231,2001,3.501522,-0.825182
3,3,5.538631,34.438397,51,3.487120,10105.0,0,ALABAMA,1624,0.000000,...,515.0,18802,0,4.976034,200112,3480,15270_20011231,2001,3.501522,-0.825182
4,4,3.767905,14.912996,3181,35.018402,230678.0,1,ALABAMA,32556,0.117217,...,18082.0,380609,0,3.825806,200112,37277,15310_20011231,2001,3.501522,-0.825182


# Data Cleaning and Wrangling

##  Includes data after 2003

In [53]:
#Time conversion
raw_data['CALLYM'] = pd.to_datetime(raw_data['CALLYM'], format='%Y%m') 
raw_data = raw_data[raw_data['CALLYM'] > pd.Timestamp('2003-01-01')]
raw_data['total_loans_equity'] = raw_data['LNLSGR'] / raw_data['EQTOT']

## Drop Columns with Too Many Missing Values

In [54]:
#Missing value detection
test = raw_data.isnull().sum()
print((test[test>0] / raw_data.shape[0]) * 100)

ILS          5.336911
ROE          0.007640
EEFFQR       0.008314
VOLIAB       0.000225
RBCT1C      60.982576
EDEPFORQ    99.749671
SCSNHAF      5.345000
ILSQ         5.345000
NIMYQ        0.000674
ASSTLT       5.345000
dtype: float64


## Reducing the Multi-collinarity within the Dataset

In [55]:
corr_matrix = raw_data.corr()

# column-pairs with corr more than 0.75
corr_pairs = []
for i, j in itertools.combinations(corr_matrix.columns, 2):
    if corr_matrix.loc[i, j] > 0.75:
        corr_pairs.append((i, j))

# number of times one specific columns appears in all pairs
col_counts = {}
for i, j in corr_pairs:
    if i in col_counts:
        col_counts[i] += 1
    else:
        col_counts[i] = 1
    if j in col_counts:
        col_counts[j] += 1
    else:
        col_counts[j] = 1

# for each pair, select the column that appears for more times
selected_cols = set()
for i, j in corr_pairs:
    if col_counts[i] > col_counts[j]:
        selected_cols.add(i)
    else:
        selected_cols.add(j)

print(selected_cols)

/var/folders/qv/gr67l17s1s7387_kfzkqk6qr0000gn/T/ipykernel_86548/2366639626.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = raw_data.corr()


{'RWAJ', 'RB2LNRES', 'LNLSGR', 'NIMQR', 'DEP', 'ILNDOMQ', 'ASSET5', 'INTINQR', 'NIMA', 'NTLNLSQ', 'ASSET2', 'LNLSNET', 'RBCT1C', 'EDEPDOMQ', 'IDT1CER', 'NETINCQR', 'ILNDOM', 'LIAB', 'ASSTLT', 'EINTEXP', 'NETINCQ', 'LNLSGRSR', 'LNLSGRS', 'NETINCR', 'DEPDOM', 'ILNDOMR', 'SZ100T3', 'INTINQ', 'EINTEXPA', 'IGLSECQR', 'EEFFQ', 'LNRE', 'EQTOT', 'EAMINTQ', 'NIMYQ', 'INTINC', 'ASSET', 'ILSQ', 'RWAJT'}


In [56]:
list1 = ["Unnamed: 0",'DEPDOM','EINTEXP',
                                                     'EEFF','EEFFR','INTINC','ILNDOM',
                                                     'ILNDOMR','IGLSECR','NIMA','NIMY',
                                                     'EAMINTAN','EAMINTANR','NTLNLS','NETINC',
                                                     "RBCT1C","RBCT1C", "EDEPFORQ", "SCSNHAF",
                                                     "RB2LNRES", "RB2LNRES", "P3ASSET","ASSTLT",
                                                     "ILS", "ILSQ","EEFFQR", "NIMYQ", "ROE",
                                                     "RWAJ", "RWAJT", "VOLIAB"]
list2 = ['DEPDOM','EINTEXP','EEFF','EEFFR','INTINC','ILNDOM','ILNDOMR','IGLSECR','NIMA','NIMY','EAMINTAN','EAMINTANR','NTLNLS','NETINC']
list3 = ['EDEP','LNLSGRS','EEFFQ','LNLSGRSR','ASSET5','ILNDOMQ','INTINQ','RB2LNRES','ASSET2','DEP','RWAJ','NIMQR','LNLSNET','LIAB','LNRE','ASSTLT']
list4 = selected_cols
union_list = list(set(list1) | set(list2) | set(list3) | set(list4) ) 
print(union_list)
data_subset = raw_data[raw_data.columns.difference(union_list)]

['RWAJ', 'LNLSGR', 'DEP', 'ASSET5', 'NTLNLSQ', 'LNLSNET', 'ASSET2', 'EDEPDOMQ', 'IDT1CER', 'NETINCQR', 'VOLIAB', 'ILNDOM', 'LIAB', 'ROE', 'EINTEXP', 'ASSTLT', 'NETINCQ', 'EAMINTANR', 'EAMINTAN', 'LNLSGRS', 'NETINCR', 'DEPDOM', 'ILNDOMR', 'EEFFQR', 'NETINC', 'NIMY', 'EINTEXPA', 'LNRE', 'EQTOT', 'EAMINTQ', 'NIMYQ', 'EDEPFORQ', 'P3ASSET', 'ASSET', 'ILS', 'EEFF', 'RB2LNRES', 'NIMQR', 'ILNDOMQ', 'INTINQR', 'IGLSECR', 'NIMA', 'RBCT1C', 'EEFFR', 'NTLNLS', 'LNLSGRSR', 'Unnamed: 0', 'SZ100T3', 'INTINQ', 'IGLSECQR', 'EEFFQ', 'EDEP', 'INTINC', 'SCSNHAF', 'ILSQ', 'RWAJT']


In [57]:
test = data_subset.isnull().sum()
print((test[test>0] / raw_data.shape[0]) * 100)
data_subset.sort_values(by=["CALLYM","CERT"],inplace = True)
data_subset.drop(['CB','CALLYM','ID'],axis=1,inplace=True)

Series([], dtype: float64)


/var/folders/qv/gr67l17s1s7387_kfzkqk6qr0000gn/T/ipykernel_86548/2494697861.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset.sort_values(by=["CALLYM","CERT"],inplace = True)
/var/folders/qv/gr67l17s1s7387_kfzkqk6qr0000gn/T/ipykernel_86548/2494697861.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset.drop(['CB','CALLYM','ID'],axis=1,inplace=True)


# Generate Dummy Variables for the State Where the Bank Exists

In [58]:
st_subset = data_subset["STNAME"]
st_subset = st_subset.str.replace(" ","_")
st_dummies = pd.get_dummies(data = st_subset)
df = pd.concat([data_subset, st_dummies], axis = 1)

# Measuring the IR Impact

## Preparing Data for Measuring the IR Impact

In [ ]:
data_subset.drop('STNAME',axis =1 , inplace = True)

cert_values_train = data_subset['CERT'].unique()
cert_sample_train = pd.Series(cert_values_train).sample(frac=0.80, random_state=422)
df_train = data_subset[data_subset['CERT'].isin(cert_sample_train)]


cert_values_test = set(cert_values_train) - set(cert_sample_train)
cert_sample_test = pd.Series(list(cert_values_test)).sample(frac=0.20, random_state=422)
df_test = data_subset[data_subset['CERT'].isin(cert_sample_test)]

assert not set(df_train['CERT']).intersection(set(df_test['CERT']))

In [60]:
columns_stra = '+'.join(data_subset.columns.str.replace(' ', '_').tolist())
columns_str = columns_stra.replace('+C(CERT)', '').replace('+C(YEAR)', '').replace('1 +','')
formula_without_y = ' ~  C(YEAR) + ' + columns_str

## Measuring the Impact of IR with Time Cluster SE

In [62]:
for y in ['SCMTGBKR', 'LNATRES' ,'RB2LNRESR', 'RBC1AAJ' ,'ILNDOMQR', 'LNLSNETR',
 'ASSTLTR', 'ROA' ,'total_loans_equity']:
    formula_final = formula(formula_without_y,y)
    formula_final = formula_final.replace(  y+'+', '')
    formula_final = formula_final.replace(  '+'+y, '')
    model = sm.GLM.from_formula(formula=formula_final, data=df_train, family=sm.families.Gaussian())
    result = model.fit(cov_type='cluster', cov_kwds={'groups': df_train['YEAR']})
    print(result.summary(alpha=0.1))
    
      # Make predictions using testing data
    y_pred = result.predict(df_test)
    y_true = df_test[y]
    
    # Evaluate model performance
    mse, r2 = evaluate_model(y_true, y_pred)
    print("Results for dependent variable:", y)
    print("MSE:", mse)
    print("R-squared:", r2)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               SCMTGBKR   No. Observations:               354888
Model:                            GLM   Df Residuals:                   354835
Model Family:                Gaussian   Df Model:                           52
Link Function:               identity   Scale:                          47.858
Method:                          IRLS   Log-Likelihood:            -1.1899e+06
Date:                Sat, 22 Apr 2023   Deviance:                   1.6982e+07
Time:                        20:31:16   Pearson chi2:                 1.70e+07
No. Iterations:                     3   Pseudo R-squ. (CS):             0.5183
Covariance Type:              cluster                                         
                         coef    std err          z      P>|z|       [0.05       0.95]
--------------------------------------------------------------------------------------
Intercept             -2.4972   4.19

                 Generalized Linear Model Regression Results                  
Dep. Variable:              RB2LNRESR   No. Observations:               354888
Model:                            GLM   Df Residuals:                   354835
Model Family:                Gaussian   Df Model:                           52
Link Function:               identity   Scale:                        0.041865
Method:                          IRLS   Log-Likelihood:                 59545.
Date:                Sat, 22 Apr 2023   Deviance:                       14855.
Time:                        20:31:41   Pearson chi2:                 1.49e+04
No. Iterations:                     3   Pseudo R-squ. (CS):             0.5309
Covariance Type:              cluster                                         
                         coef    std err          z      P>|z|       [0.05       0.95]
--------------------------------------------------------------------------------------
Intercept             -0.0074       

                 Generalized Linear Model Regression Results                  
Dep. Variable:               ILNDOMQR   No. Observations:               354888
Model:                            GLM   Df Residuals:                   354835
Model Family:                Gaussian   Df Model:                           52
Link Function:               identity   Scale:                         0.36442
Method:                          IRLS   Log-Likelihood:            -3.2442e+05
Date:                Sat, 22 Apr 2023   Deviance:                   1.2931e+05
Time:                        20:32:06   Pearson chi2:                 1.29e+05
No. Iterations:                     3   Pseudo R-squ. (CS):             0.9773
Covariance Type:              cluster                                         
                         coef    std err          z      P>|z|       [0.05       0.95]
--------------------------------------------------------------------------------------
Intercept              0.0604       

                 Generalized Linear Model Regression Results                  
Dep. Variable:                ASSTLTR   No. Observations:               354888
Model:                            GLM   Df Residuals:                   354835
Model Family:                Gaussian   Df Model:                           52
Link Function:               identity   Scale:                          225.10
Method:                          IRLS   Log-Likelihood:            -1.4647e+06
Date:                Sat, 22 Apr 2023   Deviance:                   7.9874e+07
Time:                        20:32:30   Pearson chi2:                 7.99e+07
No. Iterations:                     3   Pseudo R-squ. (CS):             0.3726
Covariance Type:              cluster                                         
                         coef    std err          z      P>|z|       [0.05       0.95]
--------------------------------------------------------------------------------------
Intercept              4.1993       

                 Generalized Linear Model Regression Results                  
Dep. Variable:     total_loans_equity   No. Observations:               354888
Model:                            GLM   Df Residuals:                   354835
Model Family:                Gaussian   Df Model:                           52
Link Function:               identity   Scale:                          833.78
Method:                          IRLS   Log-Likelihood:            -1.6970e+06
Date:                Sat, 22 Apr 2023   Deviance:                   2.9585e+08
Time:                        20:32:55   Pearson chi2:                 2.96e+08
No. Iterations:                     3   Pseudo R-squ. (CS):           0.007829
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|       [0.05       0.95]
-----------------------------------------------------------------------------------
Intercept          -3.4239   1.26e+04     

# Prediction

## Preparing Data for Predicting

In [ ]:
df.drop('STNAME',axis =1 , inplace = True)
cert_values_train = df['CERT'].unique()
cert_sample_train = pd.Series(cert_values_train).sample(frac=0.80, random_state=422)
df_train = df[df['CERT'].isin(cert_sample_train)]


cert_values_test = set(cert_values_train) - set(cert_sample_train)
cert_sample_test = pd.Series(list(cert_values_test)).sample(frac=0.20, random_state=422)
df_test = df[df['CERT'].isin(cert_sample_test)]
assert not set(df_train['CERT']).intersection(set(df_test['CERT']))

In [67]:
for y in ['SCMTGBKR', 'LNATRES' ,'RB2LNRESR', 'RBC1AAJ' ,'ILNDOMQR', 'LNLSNETR',
 'ASSTLTR', 'ROA' ,'total_loans_equity']:
    formula_final = formula(formula_without_y,y)
    formula_final = formula_final.replace(  y+'+', '')
    formula_final = formula_final.replace(  '+'+y, '')
    model = sm.GLM.from_formula(formula=formula_final, data=df_train, family=sm.families.Gaussian())
    result = model.fit(cov_type='cluster', cov_kwds={'groups': df_train['YEAR']})
    print(result.summary(alpha=0.1))
    
     # Make predictions using testing data
    y_pred = result.predict(df_test)
    y_true = df_test[y]
    
    # Evaluate model performance
    mse, r2 = evaluate_model(y_true, y_pred)
    print("Results for dependent variable:", y)
    print("MSE:", mse)
    print("R-squared:", r2)
    
    

                 Generalized Linear Model Regression Results                  
Dep. Variable:               SCMTGBKR   No. Observations:               354888
Model:                            GLM   Df Residuals:                   354835
Model Family:                Gaussian   Df Model:                           52
Link Function:               identity   Scale:                          47.858
Method:                          IRLS   Log-Likelihood:            -1.1899e+06
Date:                Sat, 22 Apr 2023   Deviance:                   1.6982e+07
Time:                        20:40:40   Pearson chi2:                 1.70e+07
No. Iterations:                     3   Pseudo R-squ. (CS):             0.5183
Covariance Type:              cluster                                         
                         coef    std err          z      P>|z|       [0.05       0.95]
--------------------------------------------------------------------------------------
Intercept             -2.4972   4.19

                 Generalized Linear Model Regression Results                  
Dep. Variable:              RB2LNRESR   No. Observations:               354888
Model:                            GLM   Df Residuals:                   354835
Model Family:                Gaussian   Df Model:                           52
Link Function:               identity   Scale:                        0.041865
Method:                          IRLS   Log-Likelihood:                 59545.
Date:                Sat, 22 Apr 2023   Deviance:                       14855.
Time:                        20:41:05   Pearson chi2:                 1.49e+04
No. Iterations:                     3   Pseudo R-squ. (CS):             0.5309
Covariance Type:              cluster                                         
                         coef    std err          z      P>|z|       [0.05       0.95]
--------------------------------------------------------------------------------------
Intercept             -0.0074       

                 Generalized Linear Model Regression Results                  
Dep. Variable:               ILNDOMQR   No. Observations:               354888
Model:                            GLM   Df Residuals:                   354835
Model Family:                Gaussian   Df Model:                           52
Link Function:               identity   Scale:                         0.36442
Method:                          IRLS   Log-Likelihood:            -3.2442e+05
Date:                Sat, 22 Apr 2023   Deviance:                   1.2931e+05
Time:                        20:41:30   Pearson chi2:                 1.29e+05
No. Iterations:                     3   Pseudo R-squ. (CS):             0.9773
Covariance Type:              cluster                                         
                         coef    std err          z      P>|z|       [0.05       0.95]
--------------------------------------------------------------------------------------
Intercept              0.0604       

                 Generalized Linear Model Regression Results                  
Dep. Variable:                ASSTLTR   No. Observations:               354888
Model:                            GLM   Df Residuals:                   354835
Model Family:                Gaussian   Df Model:                           52
Link Function:               identity   Scale:                          225.10
Method:                          IRLS   Log-Likelihood:            -1.4647e+06
Date:                Sat, 22 Apr 2023   Deviance:                   7.9874e+07
Time:                        20:41:56   Pearson chi2:                 7.99e+07
No. Iterations:                     3   Pseudo R-squ. (CS):             0.3726
Covariance Type:              cluster                                         
                         coef    std err          z      P>|z|       [0.05       0.95]
--------------------------------------------------------------------------------------
Intercept              4.1993       

                 Generalized Linear Model Regression Results                  
Dep. Variable:     total_loans_equity   No. Observations:               354888
Model:                            GLM   Df Residuals:                   354835
Model Family:                Gaussian   Df Model:                           52
Link Function:               identity   Scale:                          833.78
Method:                          IRLS   Log-Likelihood:            -1.6970e+06
Date:                Sat, 22 Apr 2023   Deviance:                   2.9585e+08
Time:                        20:42:21   Pearson chi2:                 2.96e+08
No. Iterations:                     3   Pseudo R-squ. (CS):           0.007829
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|       [0.05       0.95]
-----------------------------------------------------------------------------------
Intercept          -3.4239   1.26e+04     